In [6]:
!pip install fuzzywuzzy

In [7]:
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from scipy.stats import skew, kurtosis
import gensim
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import scipy
from tqdm import tqdm_notebook
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
! rm -r 558-project/
!git clone https://github.com/TYTYTYTYTYTYTYTYTY/558-project.git
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

Cloning into '558-project'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 47 (delta 16), reused 43 (delta 12), pack-reused 0
Unpacking objects: 100% (47/47), done.
--2019-11-07 02:05:27--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.108.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.108.109|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 1647046227 (1.5G), 1367029602 (1.3G) remaining [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[+++================>]   1.53G  62.9MB/s    in 23s     

2019-11-07 02:05:50 (57.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np 
train = pd.read_csv("558-project/data/train.csv")
test = pd.read_csv("558-project/data/test.csv")

#train = train.drop("id",1, inplace=True)
#test = test.drop("id",1, inplace=True)
print(train.head(5))


   id  ... same_source
0   1  ...           1
1   2  ...           0
2   3  ...           0
3   4  ...           0
4   5  ...           0

[5 rows x 4 columns]


In [10]:
model = gensim.models.KeyedVectors.load_word2vec_format( \
    'GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
def sent2vec(s,model):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

#sent1 vector
train_sent1_vectors = np.zeros((train.shape[0], 300))
for i, s in enumerate(tqdm_notebook(train.sent1.values)):
    train_sent1_vectors[i, :] = sent2vec(s,model)
    
#sent2 vector    
train_sent2_vectors  = np.zeros((train.shape[0], 300))
for i, s in enumerate(tqdm_notebook(train.sent2.values)):
    train_sent2_vectors[i, :] = sent2vec(s,model) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [12]:
#sent1 vector
test_sent1_vectors = np.zeros((test.shape[0], 300))
for i, s in enumerate(tqdm_notebook(test.sent1.values)):
    test_sent1_vectors[i, :] = sent2vec(s,model)
    
#sent2 vector    
test_sent2_vectors  = np.zeros((test.shape[0], 300))
for i, s in enumerate(tqdm_notebook(test.sent2.values)):
    test_sent2_vectors[i, :] = sent2vec(s,model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [13]:
train['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]
train['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(train_sent1_vectors), np.nan_to_num(train_sent2_vectors))]




test['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]
test['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(test_sent1_vectors), np.nan_to_num(test_sent2_vectors))]

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:1178: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [0]:
train['skew_s1vec'] = [skew(x) for x in np.nan_to_num(train_sent1_vectors)]
train['skew_s2vec'] = [skew(x) for x in np.nan_to_num(train_sent2_vectors)]
train['kur_s1vec'] = [kurtosis(x) for x in np.nan_to_num(train_sent1_vectors)]
train['kur_s2vec'] = [kurtosis(x) for x in np.nan_to_num(train_sent2_vectors)]


test['skew_s1vec'] = [skew(x) for x in np.nan_to_num(test_sent1_vectors)]
test['skew_s2vec'] = [skew(x) for x in np.nan_to_num(test_sent2_vectors)]
test['kur_s1vec'] = [kurtosis(x) for x in np.nan_to_num(test_sent1_vectors)]
test['kur_s2vec'] = [kurtosis(x) for x in np.nan_to_num(test_sent2_vectors)]


In [15]:
train_ckpt_distance = train[['cosine_distance','cityblock_distance','jaccard_distance',
                'canberra_distance','euclidean_distance','minkowski_distance','braycurtis_distance',
                 'skew_s1vec','skew_s2vec','kur_s1vec','kur_s2vec']]
print(train_ckpt_distance.shape)



test_ckpt_distance = test[['cosine_distance','cityblock_distance','jaccard_distance',
                'canberra_distance','euclidean_distance','minkowski_distance','braycurtis_distance',
                 'skew_s1vec','skew_s2vec','kur_s1vec','kur_s2vec']]
print(test_ckpt_distance.shape)

del(model)

(129156, 11)
(14350, 11)


In [0]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)

In [17]:
model = gensim.models.KeyedVectors.load_word2vec_format( \
    'GoogleNews-vectors-negative300.bin.gz', binary=True)

train['wmd'] = train.apply(lambda x: wmd(x['sent1'], x['sent2']), axis=1)

test['wmd'] = test.apply(lambda x: wmd(x['sent1'], x['sent2']), axis=1)

del(model)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
norm_model = gensim.models.KeyedVectors.load_word2vec_format( \
    'GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)

train['norm_wmd'] = train.apply(lambda x: norm_wmd(x['sent1'], x['sent2']), axis=1)

test['norm_wmd'] = test.apply(lambda x: norm_wmd(x['sent1'], x['sent2']), axis=1)

del(norm_model)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
train_ckpt_wmd=train[['wmd','norm_wmd']]
train_ckpt_wmd

test_ckpt_wmd=test[['wmd','norm_wmd']]
test_ckpt_wmd

,wmd,norm_wmd
0,3.234672,1.289266
1,3.363726,1.239698
2,4.202454,1.360028
3,4.247422,1.341462
4,3.347305,1.280948
...,...,...
14345,3.681080,1.334166
14346,3.670352,1.341901
14347,3.405346,1.288707
14348,3.574445,1.313100


## embedding matrix 


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [21]:
texts=pd.concat([train.sent1, train.sent2], ignore_index= True)

print(texts.shape)

NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_sent1_train = tokenizer.texts_to_sequences(train.sent1)
sequences_sent2_train = tokenizer.texts_to_sequences(train.sent2)
sequences_sent1_test = tokenizer.texts_to_sequences(test.sent1)
sequences_sent2_test = tokenizer.texts_to_sequences(test.sent2)
sequences_train = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))



(258312,)
Found 140450 unique tokens.


In [22]:
sequences_train = pad_sequences(sequences_train)
pad_sent1_train,pad_sent2_train = train_test_split(sequences_train,test_size=0.5, shuffle= False )
pad_sent1_test = pad_sequences(sequences_sent1_test,maxlen= sequences_train.shape[1])
pad_sent2_test = pad_sequences(sequences_sent2_test,maxlen= sequences_train.shape[1])

print(pad_sent1_train.shape)
print(pad_sent2_train.shape)
print(pad_sent1_test.shape)
print(pad_sent2_test.shape)

print(pad_sent1_train[8,:])


(129156, 167)
(129156, 167)
(14350, 167)
(14350, 167)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    1   28   76    7 4101   21   52   95   99  847   53   93]


In [23]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [24]:
print(embedding_matrix.shape)

(140451, 300)


In [25]:
mm_train = np.hstack((train_ckpt_wmd,train_ckpt_distance))
mm_train.shape

mm_test = np.hstack((test_ckpt_wmd,test_ckpt_distance))
mm_test.shape



(14350, 13)

In [0]:
import numpy as np
import pandas as pd
np.random.seed(0)

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, BatchNormalization,concatenate,Subtract, Dot, Multiply,Bidirectional,Lambda
from keras.layers.embeddings import Embedding
from keras.initializers import glorot_uniform
from keras.layers.noise import GaussianNoise
from keras import backend as K
from keras import optimizers
import tensorflow as tf

import keras.callbacks as kcallbacks
np.random.seed(1)

import warnings
warnings.filterwarnings('ignore')


In [0]:
def get_model():
    embedding_layer = Embedding(vocabulary_size,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=167,
                                trainable=False)

    sent_1 = Input(shape=(167,), dtype='int32')
    y1 = embedding_layer(sent_1)
    

    sent_2 = Input(shape=(167,), dtype='int32')
    y2 = embedding_layer(sent_2)
   
    
    shared_lstm_1 = LSTM(75, return_sequences=True)
    shared_lstm_2 = LSTM(75)

    q1 = shared_lstm_1(y1)
    q1 = Dropout(0.5)(q1)
    q1 = BatchNormalization()(q1)
    q1 = shared_lstm_2(q1)
        # q1 = Dropout(0.5)(q1)

    q2 = shared_lstm_1(y2)
    q2 = Dropout(0.5)(q2)
    q2 = BatchNormalization()(q2)
    q2 = shared_lstm_2(q2)

    d = Subtract()([q1, q2])
    
    distance = Multiply()([d, d])

    angle = Multiply()([q1, q2])

    magic_input = Input(shape=(13,))
    magic_dense = BatchNormalization()(magic_input)
    magic_dense = Dense(64, activation='relu')(magic_dense)
        #magic_dense = Dropout(0.3)(magic_dense)
        
    merged = concatenate([distance,angle,magic_dense])
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged)

    merged = Dense(256, activation='relu')(merged)  # 64
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged)

    merged = Dense(64, activation='relu')(merged)  # 64
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged)

    is_duplicate = Dense(1, activation='sigmoid')(merged)

    model = Model(inputs=[sent_1, sent_2, magic_input], outputs=is_duplicate)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()
    return model


In [28]:
model = get_model()











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 167)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 167)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 167, 300)     42135300    input_1[0][0]                    
                                                                 inpu

In [0]:
mm_s_train, mm_val = train_test_split(mm_train, test_size = 0.01, random_state = 12357)
pad_s_sent1_train, pad_sent1_val = train_test_split(pad_sent1_train, test_size = 0.01, random_state = 12357)
pad_s_sent2_train, pad_sent2_val = train_test_split(pad_sent2_train, test_size = 0.01, random_state = 12357)
y_s_train, y_val = train_test_split(train.same_source, test_size = 0.01, random_state = 12357)

In [35]:
early_stopping = kcallbacks.EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = 'best_model.h5'
model_checkpoint = kcallbacks.ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([pad_s_sent1_train, pad_s_sent2_train,mm_s_train], y_s_train, \
                 validation_data=([pad_sent1_val, pad_sent2_val, mm_val], y_val), \
                 epochs=50, batch_size=256, shuffle=True, callbacks=[early_stopping, model_checkpoint])
predicts = model.predict([pad_sent1_test, pad_sent2_test,mm_test], batch_size=10, verbose=1)


Train on 127864 samples, validate on 1292 samples
Epoch 1/50
127864/127864 [==============================] - 578s 5ms/step - loss: 0.5585 - acc: 0.7079 - val_loss: 0.5389 - val_acc: 0.7229
Epoch 2/50
127864/127864 [==============================] - 580s 5ms/step - loss: 0.5381 - acc: 0.7244 - val_loss: 0.5152 - val_acc: 0.7322
Epoch 3/50
127864/127864 [==============================] - 579s 5ms/step - loss: 0.5228 - acc: 0.7361 - val_loss: 0.5000 - val_acc: 0.7500
Epoch 4/50
127864/127864 [==============================] - 581s 5ms/step - loss: 0.5100 - acc: 0.7438 - val_loss: 0.5066 - val_acc: 0.7415
Epoch 5/50
127864/127864 [==============================] - 580s 5ms/step - loss: 0.4969 - acc: 0.7535 - val_loss: 0.4893 - val_acc: 0.7531
Epoch 6/50
127864/127864 [==============================] - 569s 4ms/step - loss: 0.4861 - acc: 0.7610 - val_loss: 0.4874 - val_acc: 0.7585
Epoch 7/50
127864/127864 [==============================] - 566s 4ms/step - loss: 0.4768 - acc: 0.7669 - val_l

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [37]:
!ls /content/gdrive/My\ Drive/

 558project
'Art history 6a.docx'
'Colab Notebooks'
 ess9.gslides
'Gardner&#39%3s+Art+through+the+Ages+The+Weste....pdf'
'Geog W12 Lab 7 Website'
'Glob notes'
'https:  support.google.com installer answer su.png'
 IMG_4724.JPG
 pastmidterm_sol.gdoc
'“Places I’ve Visited in California.gtable'
 projects
 Untitled0.ipynb
'Welcome back, Kimi! | Scuderia Ferrari (1).png'
'Welcome back, Kimi! | Scuderia Ferrari (2).png'
'Welcome back, Kimi! | Scuderia Ferrari.png'
'作死大旅游 MK II.gslides'
 圣诞作死大旅游.gslides
 未命名的表单.gform


In [38]:
print(predicts)

[[0.02444449]
 [0.9795735 ]
 [0.02872616]
 ...
 [0.9616339 ]
 [0.08862007]
 [0.879396  ]]


In [0]:
root_path = '/content/gdrive/My Drive/'

out= pd.DataFrame()
out['id'] = test['id']
out['same_source'] = predicts

out.to_csv(root_path+'output.csv', index = False)

